## Import Dependencies

In [24]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

## Load Data

In [25]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

## Split Data

In [26]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

In [27]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

## Define Alexnet Model

In [28]:
model = models.alexnet(pretrained=True)

In [29]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [30]:
device = torch.device('cuda')
model = model.to(device)

## Live Plotting

In [31]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.tickers import SingleIntervalTicker
output_notebook()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

p1 = figure(title="Loss", x_axis_label="Epoch", plot_height=300, plot_width=360)
p2 = figure(title="Accuracy", x_axis_label="Epoch", plot_height=300, plot_width=360)

source1 = ColumnDataSource(data={'epochs': [], 'trainlosses': [], 'testlosses': [] })
source2 = ColumnDataSource(data={'epochs': [], 'train_accuracies': [], 'test_accuracies': []})

#r = p1.multi_line(ys=['trainlosses', 'testlosses'], xs='epochs', color=colors, alpha=0.8, legend_label=['Training','Test'], source=source)
r1 = p1.line(x='epochs', y='trainlosses', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source1)
r2 = p1.line(x='epochs', y='testlosses', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source1)

r3 = p2.line(x='epochs', y='train_accuracies', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source2)
r4 = p2.line(x='epochs', y='test_accuracies', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source2)

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p2.legend.location = "bottom_right"
p2.legend.click_policy="hide"

Loading BokehJS ...

## Train

In [34]:
BEST_MODEL_PATH = 'best_model_alex_new.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

handle = show(row(p1, p2), notebook_handle=True)

for epoch in range(70):
    
    train_loss = 0.0
    train_error_count = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        train_loss += loss
        train_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    test_loss = 0.0
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        test_loss += loss
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    test_loss /= len(test_loader)
    
    train_accuracy = 1.0 - float(train_error_count) / float(len(train_dataset))
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f, %f, %f, %f' % (epoch+1, train_loss, test_loss, train_accuracy, test_accuracy))
    
    
    new_data1 = {'epochs': [epoch+1],
                 'trainlosses': [float(train_loss)],
                 'testlosses': [float(test_loss)] }
    source1.stream(new_data1)
    new_data2 = {'epochs': [epoch+1],
                 'train_accuracies': [float(train_accuracy)],
                 'test_accuracies': [float(test_accuracy)] }
    source2.stream(new_data2)
    push_notebook(handle=handle)
    
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

1: 0.518040, 0.224027, 0.709677, 0.920000
2: 0.198140, 0.189834, 0.927419, 0.880000
3: 0.138011, 0.132873, 0.943548, 0.940000
4: 0.086605, 0.361028, 0.975806, 0.900000
5: 0.108686, 0.219835, 0.975806, 0.960000
6: 0.059240, 0.166224, 0.975806, 0.960000
7: 0.062118, 0.066924, 0.983871, 0.960000
8: 0.027868, 0.082478, 0.991935, 0.980000
9: 0.016477, 0.073025, 0.991935, 0.960000
10: 0.012299, 0.063433, 1.000000, 0.980000
11: 0.007386, 0.072676, 1.000000, 0.960000
12: 0.006155, 0.038997, 1.000000, 0.980000
13: 0.007690, 0.070820, 0.991935, 0.980000
14: 0.004344, 0.068904, 1.000000, 0.980000
15: 0.004853, 0.076878, 1.000000, 0.980000
16: 0.004186, 0.052526, 1.000000, 0.980000
17: 0.001410, 0.167414, 1.000000, 0.960000
18: 0.002356, 0.057224, 1.000000, 0.980000
19: 0.007754, 0.052841, 1.000000, 0.960000
20: 0.006895, 0.114489, 0.991935, 0.980000
21: 0.013980, 0.034274, 0.991935, 0.980000
22: 0.001730, 0.072627, 1.000000, 0.980000
23: 0.001784, 0.030121, 1.000000, 0.980000
24: 0.004325, 0.0363

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Testing loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to compute gradients during inference
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        predicted_labels.extend(outputs.argmax(1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate metrics
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

## Define resent model

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
model.fc = torch.nn.Linear(512, 2)

In [ ]:
device = torch.device('cuda')
model = model.to(device)

## Train Resnet

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model_resnet18.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

## Define Googenet

In [17]:
# Load pretrained FaceNet model
model = InceptionResnetV1(pretrained='vggface2', classify=True)

model = models.alexnet(pretrained=True)

# Modify the classification layer for your specific task
model.logits = nn.Linear(512, 2)  # Assuming you want 2 classes

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

NameError: name 'InceptionResnetV1' is not defined

In [ ]:
model = models.googlenet(pretrained=True)

In [ ]:
model.fc = torch.nn.Linear(1024, 2)

In [ ]:
device = torch.device('cuda')
model = model.to(device)

## Train Model

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model_googlenet.pth'
best_accuracy = 0.0

# Load pretrained GoogleNet model
model = models.googlenet(pretrained=True)

# Modify the classification layer for your specific task
model.fc = torch.nn.Linear(1024, 2)  # Assuming you want 2 classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    test_error_count = 0.0
    total_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += torch.sum(outputs.argmax(1) != labels)
        total_samples += len(labels)
    
    test_accuracy = 1.0 - float(test_error_count) / float(total_samples)
    print('%d: %f' % (epoch, test_accuracy))
    
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100.0%
